In [41]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
from sqlalchemy import join
from sqlalchemy import select
import random
import string

In [14]:
#calling sqlalchemy declerative base for models
Base = declarative_base()

<ipython-input-14-14385c17bab9>:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [15]:
#creating models
class User(Base):
    __tablename__ = 'users'
    __table_args__ = {'extend_existing': True}


    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True, nullable=False)
    email = Column(String, unique=True, nullable=False)

    comments = relationship('Comment', back_populates='user')

class Comment(Base):
    __tablename__ = 'comments'
    __table_args__ = {'extend_existing': True}


    id = Column(Integer, primary_key=True)
    text = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship('User', back_populates='comments')

In [16]:
#create an engine for e.g: sqlite ... db connection
engine = create_engine('sqlite:///example.db')

In [22]:
#Base.metadata.create_all(engine) creates all the tables defined in the Base
#class in the database specified by the engine.
# In this case, it will create the 'users' and 'comments' tables in the 'example.db' SQLite database.

Base.metadata.create_all(engine)

In [23]:
#binding session
Session = sessionmaker(bind=engine)
session = Session()

In [24]:
#persisting the db
try:
    for i in range(1, 11):
        email = f'user{i}' + ''.join(random.choices(string.ascii_lowercase, k=5)) + '@example.com'
        user = User(username=f'user{i}', email=email)
        session.add(user)

    session.commit()

    for user in session.query(User).all():
        for i in range(1, 4):
            comment = Comment(text=f'Comment {i} by {user.username}', user_id=user.id)
            session.add(comment)

    session.commit()

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()

An error occurred: (sqlite3.IntegrityError) UNIQUE constraint failed: users.username
[SQL: INSERT INTO users (username, email) VALUES (?, ?) RETURNING id]
[parameters: ('user1', 'user1snicp@example.com')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [55]:
#simple select users
users = session.query(User).all()
for user in users:
    print(f"User ID: {user.id}, Username: {user.username}, Email: {user.email}, Comments: {len(user.comments)}")

User ID: 1, Username: user1, Email: user1@example.com, Comments: 8
User ID: 2, Username: user2, Email: user2@example.com, Comments: 3
User ID: 3, Username: user3, Email: user3@example.com, Comments: 3
User ID: 4, Username: user4, Email: user4@example.com, Comments: 3
User ID: 5, Username: user5, Email: user5@example.com, Comments: 3
User ID: 6, Username: user6, Email: user6@example.com, Comments: 3
User ID: 7, Username: user7, Email: user7@example.com, Comments: 3
User ID: 8, Username: user8, Email: user8@example.com, Comments: 3
User ID: 9, Username: user9, Email: user9@example.com, Comments: 3
User ID: 10, Username: user10, Email: user10@example.com, Comments: 3
User ID: 11, Username: newtest_user, Email: test_user@example.com, Comments: 1
User ID: 12, Username: newtest1_user, Email: test_user1@example.com, Comments: 1


In [53]:
#simple select comments
comments = session.query(Comment).all()
for comment in comments:
    print(f"Comment ID: {comment.id}, Text: {comment.text}, User ID: {comment.user_id}")

Comment ID: 1, Text: Comment 1 by user1, User ID: 1
Comment ID: 2, Text: Comment 2 by user1, User ID: 1
Comment ID: 3, Text: Comment 3 by user1, User ID: 1
Comment ID: 4, Text: Comment 1 by user2, User ID: 2
Comment ID: 5, Text: Comment 2 by user2, User ID: 2
Comment ID: 6, Text: Comment 3 by user2, User ID: 2
Comment ID: 7, Text: Comment 1 by user3, User ID: 3
Comment ID: 8, Text: Comment 2 by user3, User ID: 3
Comment ID: 9, Text: Comment 3 by user3, User ID: 3
Comment ID: 10, Text: Comment 1 by user4, User ID: 4
Comment ID: 11, Text: Comment 2 by user4, User ID: 4
Comment ID: 12, Text: Comment 3 by user4, User ID: 4
Comment ID: 13, Text: Comment 1 by user5, User ID: 5
Comment ID: 14, Text: Comment 2 by user5, User ID: 5
Comment ID: 15, Text: Comment 3 by user5, User ID: 5
Comment ID: 16, Text: Comment 1 by user6, User ID: 6
Comment ID: 17, Text: Comment 2 by user6, User ID: 6
Comment ID: 18, Text: Comment 3 by user6, User ID: 6
Comment ID: 19, Text: Comment 1 by user7, User ID: 7
Co

In [52]:
#select with join
comments_with_users = session.query(Comment, User).join(User).all()
for comment, user in comments_with_users:
    print(f"Comment ID: {comment.id}, Text: {comment.text}, User ID: {user.id}, Username: {user.username}, Email: {user.email}")


Comment ID: 1, Text: Comment 1 by user1, User ID: 1, Username: user1, Email: user1@example.com
Comment ID: 2, Text: Comment 2 by user1, User ID: 1, Username: user1, Email: user1@example.com
Comment ID: 3, Text: Comment 3 by user1, User ID: 1, Username: user1, Email: user1@example.com
Comment ID: 4, Text: Comment 1 by user2, User ID: 2, Username: user2, Email: user2@example.com
Comment ID: 5, Text: Comment 2 by user2, User ID: 2, Username: user2, Email: user2@example.com
Comment ID: 6, Text: Comment 3 by user2, User ID: 2, Username: user2, Email: user2@example.com
Comment ID: 7, Text: Comment 1 by user3, User ID: 3, Username: user3, Email: user3@example.com
Comment ID: 8, Text: Comment 2 by user3, User ID: 3, Username: user3, Email: user3@example.com
Comment ID: 9, Text: Comment 3 by user3, User ID: 3, Username: user3, Email: user3@example.com
Comment ID: 10, Text: Comment 1 by user4, User ID: 4, Username: user4, Email: user4@example.com
Comment ID: 11, Text: Comment 2 by user4, User ID

In [37]:
#add new user
try:
    new_user = User(username='test_user', email='test_user1@example.com')
    session.add(new_user)
    session.commit()

    new_comment = Comment(text='This is a new comment testing.', user_id=new_user.id)
    session.add(new_comment)
    session.commit()

    user_with_comment = session.query(User, Comment).join(Comment).filter(User.id == new_user.id).first()
    user, comment = user_with_comment
    print(f"User ID: {user.id}, Username: {user.username}, Email: {user.email}")
    print(f"Comment ID: {comment.id}, Text: {comment.text}")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()

User ID: 12, Username: test_user, Email: test_user1@example.com
Comment ID: 32, Text: This is a new comment testing.


In [40]:
#update user
try:
    user_to_update = session.query(User).filter(User.username == 'test_user').first()

    if user_to_update:
        user_to_update.username = 'newtest1_user'
        session.commit()
        print("Username updated successfully!")
    else:
        print("User not found!")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()

Username updated successfully!


In [51]:
name = 'user1'
try:
    stmt_user = select(User).where(User.username == name)
    example_user = session.execute(stmt_user).scalar()

    if example_user:
        new_comment = Comment(text="This is a new comment for make_change_doc.", user_id=example_user.id)
        session.add(new_comment)
        session.commit()
        print("Changes committed successfully for "+name)
    else:
        print("User  name not found."+name)

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()


Changes committed successfully for user1
